In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import os

USE_DRIVE = False
if USE_DRIVE:
    from google.colab import drive

    drive.mount('/content/gdrive')
    ROOT_DRIVE = '/content/gdrive/MyDrive/'
    DRIVE_FOLDER_PATH = os.path.join(ROOT_DRIVE, 'HuBMAP')
    IMAGES_PATH = os.path.join(DRIVE_FOLDER_PATH, 'hubmap-256x256', 'train')
    MASKS_PATH = os.path.join(DRIVE_FOLDER_PATH, 'hubmap-256x256', 'masks')
else:
    IMAGES_PATH = os.path.join('data', '256x256', 'train')
    MASKS_PATH = os.path.join('data', '256x256', 'masks')

In [3]:
!nvidia-smi

fish: Unknown command: nvidia-smi
fish: 
nvidia-smi
^


# DeepLab HuBMAP

In [4]:
from utils import get_device_colab, set_deterministic_colab

NOT_COLAB = True
if not NOT_COLAB:

    DEVICE = get_device_colab()
else:
    DEVICE = 'cpu'

SEED = 42
set_deterministic_colab(SEED)

In [5]:
from model.pretrained_models import get_deeplabv3_resnet

model = get_deeplabv3_resnet(DEVICE, resnet_layers=101).to(DEVICE)

In [6]:
print(model)

DeepLabV3(
  (backbone): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Se

In [7]:
from preprocessing.dataset import get_training_validation_sets, HuBMAPDataset
from preprocessing.augmentation import get_augmentations
from training.loop import Trainer
from training.loss_functions import BinaryDiceLoss

import torch
from torchvision import transforms

# From https://pytorch.org/docs/stable/torchvision/models.html
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

training_dataset, training_images, validation_dataset, validation_images = get_training_validation_sets(IMAGES_PATH,
                                                                                                        MASKS_PATH,
                                                                                                        0.3, 
                                                                                                        {'train': get_augmentations(train=True, dihedral_p=0.0, distortion_p=0.0),
                                                                                                         'val': None},
                                                                                                        mean,
                                                                                                        std)

criterion = BinaryDiceLoss(logits=True).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

trainer = Trainer(model,
                  0.3,
                  criterion,
                  optimizer,
                  16,
                  DEVICE,
                  "logs",
                  training_dataset,
                  validation_dataset)

stats, eval_stats = trainer.train(1, verbose=True, limit=1)

/home/alessandro/University/ML4CV/HackingTheKidney/training/loop.py:227: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if weights_dir is 'dmyhms':
  0%|          | 0/161 [00:00<?, ?it/s]

Training epoch 1/1:


/home/alessandro/University/ML4CV/HackingTheKidney/venv/lib/python3.9/site-packages/torch/autograd/__init__.py:130: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  Variable._execution_engine.run_backward(
  0%|          | 0/69 [00:00<?, ?it/s]

Epoch 1 ended in 42.91151428222656 seconds
Average training batch time 42.416006326675415 seconds
Average loss 0.9394986629486084
Metrics:
Average IoU 			 0.008483671583235264
Average dice coefficient 	 0.01647220179438591
Average pixel accuracy 		 0.9508590698242188
----------------------------------------------------------------------------------------------------
Validation phase:


  1%|▏         | 1/69 [00:13<15:48, 13.95s/it]

Evaluation ended in 13.951504468917847 seconds
Average eval batch time 13.797667741775513 seconds
Average loss 0.94338059425354
Metrics:
Average IoU 			 0.09472329914569855
Average dice coefficient 	 0.14126865565776825
Average pixel accuracy 		 0.8903779983520508
----------------------------------------------------------------------------------------------------


In [8]:
print(stats)

      epoch_train_time      batch_train_time                  loss                     iou       dice_coefficient        pixel_accuracy
1  [42.91151428222656]  [42.416006326675415]  [0.9394986629486084]  [0.008483671583235264]  [0.01647220179438591]  [0.9508590698242188]


In [10]:
import pickle
stats.save('logs/stats')

with open('logs/stats', 'rb') as handle:
    b = pickle.load(handle)

print(b)

{1: {'epoch_train_time': [42.91151428222656], 'batch_train_time': [42.416006326675415], 'loss': [0.9394986629486084], 'iou': [0.008483671583235264], 'dice_coefficient': [0.01647220179438591], 'pixel_accuracy': [0.9508590698242188]}}


In [ ]:
"""
from evaluation.test import Test

tester = Test(model,
              0.3,
              "data/test",
              256,
              0,
              DEVICE)

sub_dict = tester.test("logs/submission.csv", verbose=True)

sub_dict
"""